In [1]:
import pandas as pd

# Load genetic profiles data
genetic_df = pd.read_csv("C:\\Users\\trejan\\Desktop\\Sem 2\\Machine Learning\\model\\genetic_profiles.csv")

# Load nutritional data
meal_df = pd.read_csv("C:\\Users\\trejan\\Desktop\\Sem 2\\Machine Learning\\model\\train.csv")

In [7]:
# Check genetic profiles data
print(genetic_df.head())
print(genetic_df.isnull().sum())

# Check nutritional data
print(meal_df.head())
print(meal_df.isnull().sum())

# Fill missing values for numeric columns only
numeric_cols = genetic_df.select_dtypes(include=['float64', 'int64']).columns
genetic_df[numeric_cols] = genetic_df[numeric_cols].fillna(genetic_df[numeric_cols].mean())

   Profile_ID  Age   BMI   MC4R_Variant PPARG_Variant   FTO_Variant  \
0           1   51  24.5  rs17782313_TT  rs1801282_CC  rs9939609_TT   
1           2   54  39.1  rs17782313_TT  rs1801282_CG  rs9939609_AA   
2           3   43  29.4  rs17782313_CT  rs1801282_GG  rs9939609_AT   
3           4   43  33.7  rs17782313_CT  rs1801282_CG  rs9939609_TT   
4           5   26  21.3  rs17782313_TT  rs1801282_CG  rs9939609_AT   

   LEPR_Variant  Physical_Activity  Diet_Type  Obesity_Risk_Score  
0  rs1137101_AG                  0          2                0.42  
1  rs1137101_AG                  0          1                0.40  
2  rs1137101_AA                  0          0                0.93  
3  rs1137101_AA                  0          2                0.80  
4  rs1137101_AA                  0          2                0.28  
Profile_ID            0
Age                   0
BMI                   0
MC4R_Variant          0
PPARG_Variant         0
FTO_Variant           0
LEPR_Variant         

In [8]:
# Fill missing values for categorical columns
categorical_cols = genetic_df.select_dtypes(include=['object']).columns
genetic_df[categorical_cols] = genetic_df[categorical_cols].fillna('Unknown')

In [4]:
# Features (X) and target (y)
X_genetic = genetic_df.drop(['Obesity_Risk_Score'], axis=1)
y_genetic = genetic_df['Obesity_Risk_Score']

In [9]:
print(genetic_df.isnull().sum())

Profile_ID            0
Age                   0
BMI                   0
MC4R_Variant          0
PPARG_Variant         0
FTO_Variant           0
LEPR_Variant          0
Physical_Activity     0
Diet_Type             0
Obesity_Risk_Score    0
dtype: int64


In [10]:
from sklearn.preprocessing import LabelEncoder

# Encode categorical columns
le = LabelEncoder()
for col in categorical_cols:
    genetic_df[col] = le.fit_transform(genetic_df[col])

In [11]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans

# Load datasets
genetic_df = pd.read_csv("C:\\Users\\trejan\\Desktop\\Sem 2\\Machine Learning\\model\\genetic_profiles.csv")
meal_df = pd.read_csv("C:\\Users\\trejan\\Desktop\\Sem 2\\Machine Learning\\model\\train.csv")

# Handle missing values for numeric columns in genetic_df
numeric_cols = genetic_df.select_dtypes(include=['float64', 'int64']).columns
genetic_df[numeric_cols] = genetic_df[numeric_cols].fillna(genetic_df[numeric_cols].mean())

# Handle missing values for categorical columns in genetic_df
categorical_cols = genetic_df.select_dtypes(include=['object']).columns
genetic_df[categorical_cols] = genetic_df[categorical_cols].fillna('Unknown')

# Encode categorical columns in genetic_df
le = LabelEncoder()
for col in categorical_cols:
    genetic_df[col] = le.fit_transform(genetic_df[col])

# Prepare features and target for genetic data
X_genetic = genetic_df.drop(['Obesity_Risk_Score'], axis=1)
y_genetic = genetic_df['Obesity_Risk_Score']

# Split genetic data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_genetic, y_genetic, test_size=0.2, random_state=42)

# Train a Random Forest model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Evaluate the model
y_pred = rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Preprocess nutritional data
nutritional_features = meal_df[['Energy_kcal', 'Protein_g', 'Fat_g', 'Carb_g']]
scaler = StandardScaler()
nutritional_features_scaled = scaler.fit_transform(nutritional_features)

# Cluster meals using K-Means
kmeans = KMeans(n_clusters=5, random_state=42)
meal_df['Meal_Cluster'] = kmeans.fit_predict(nutritional_features_scaled)

# Define a mapping function to map risk scores to meal clusters
def map_risk_to_cluster(risk_score):
    if risk_score < 0.3:
        return 0  # Low-risk cluster
    elif 0.3 <= risk_score < 0.6:
        return 1  # Medium-risk cluster
    else:
        return 2  # High-risk cluster

# Example new genetic profile
new_profile = pd.DataFrame({
    'BMI': [24.5],
    'MC4R_Variant': ['rs17782313_TT'],
    'PPARG_Variant': ['rs1801282_CC'],
    'FTO_Variant': ['rs9939609_TT'],
    'LEPR_Variant': ['rs1137101_AG'],
    'Physical_Activity': ['High'],
    'Diet_Type': ['High-Fat']
})

# Encode categorical variables in the new profile
new_profile = pd.get_dummies(new_profile, drop_first=True)

# Ensure the new profile has the same columns as the training data
missing_cols = set(X_train.columns) - set(new_profile.columns)
for col in missing_cols:
    new_profile[col] = 0
new_profile = new_profile[X_train.columns]

# Predict risk score for the new profile
predicted_risk = rf.predict(new_profile)
print(f'Predicted Obesity Risk Score: {predicted_risk[0]}')

# Map predicted risk score to a meal cluster
recommended_cluster = map_risk_to_cluster(predicted_risk[0])
print(f'Recommended Meal Cluster: {recommended_cluster}')

# Recommend meals from the recommended cluster
recommended_meals = meal_df[meal_df['Meal_Cluster'] == recommended_cluster]
print(recommended_meals[['Descrip', 'Energy_kcal', 'Protein_g', 'Fat_g', 'Carb_g']])

Mean Squared Error: 0.07158341387500002


C:\Users\trejan\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\trejan\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "C:\Users\trejan\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 554, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\trejan\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 

Predicted Obesity Risk Score: 0.4735000000000001
Recommended Meal Cluster: 1
                                                Descrip  Energy_kcal  \
0                          Soy flour, full-fat, roasted        441.0   
15    Fast foods, english muffin, with cheese and sa...        338.0   
19         Peanuts, virginia, oil-roasted, without salt        578.0   
20              Cheesefurter, cheese smokie, pork, beef        328.0   
26    Restaurant, family style, fried mozzarella sticks        325.0   
...                                                 ...          ...   
6869  BURGER KING, CROISSAN'WICH with Sausage and Ch...        376.0   
6874  Pork, fresh, spareribs, separable lean and fat...        397.0   
6877  Margarine-like, vegetable oil-butter spread, r...        450.0   
6879  DOMINO'S 14" Pepperoni Pizza, Classic Hand-Tos...        273.0   
6885  Beef, Australian, Wagyu, rib, small end rib st...        330.0   

      Protein_g  Fat_g  Carb_g  
0         34.80  21.86   

In [12]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score
from sklearn.cluster import KMeans

# Load datasets
genetic_df = pd.read_csv("C:\\Users\\trejan\\Desktop\\Sem 2\\Machine Learning\\model\\genetic_profiles.csv")
meal_df = pd.read_csv("C:\\Users\\trejan\\Desktop\\Sem 2\\Machine Learning\\model\\train.csv")

# Handle missing values for numeric columns in genetic_df
numeric_cols = genetic_df.select_dtypes(include=['float64', 'int64']).columns
genetic_df[numeric_cols] = genetic_df[numeric_cols].fillna(genetic_df[numeric_cols].mean())

# Handle missing values for categorical columns in genetic_df
categorical_cols = genetic_df.select_dtypes(include=['object']).columns
genetic_df[categorical_cols] = genetic_df[categorical_cols].fillna('Unknown')

# Encode categorical columns in genetic_df
le = LabelEncoder()
for col in categorical_cols:
    genetic_df[col] = le.fit_transform(genetic_df[col])

# Convert Obesity_Risk_Score into categories (low, medium, high)
genetic_df['Obesity_Risk_Category'] = pd.cut(
    genetic_df['Obesity_Risk_Score'],
    bins=[0, 0.3, 0.6, 1],
    labels=['Low', 'Medium', 'High']
)

# Prepare features and target for genetic data
X_genetic = genetic_df.drop(['Obesity_Risk_Score', 'Obesity_Risk_Category'], axis=1)
y_genetic = genetic_df['Obesity_Risk_Category']

# Split genetic data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_genetic, y_genetic, test_size=0.2, random_state=42)

# Hyperparameter tuning for Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

# Use RandomForestClassifier for classification
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(f'Best Parameters: {best_params}')

# Train the model with the best parameters
best_rf = grid_search.best_estimator_
best_rf.fit(X_train, y_train)

# Evaluate the model
y_pred = best_rf.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

# Preprocess nutritional data
nutritional_features = meal_df[['Energy_kcal', 'Protein_g', 'Fat_g', 'Carb_g']]
scaler = StandardScaler()
nutritional_features_scaled = scaler.fit_transform(nutritional_features)

# Cluster meals using K-Means
kmeans = KMeans(n_clusters=5, random_state=42)
meal_df['Meal_Cluster'] = kmeans.fit_predict(nutritional_features_scaled)

# Define a mapping function to map risk categories to meal clusters
def map_risk_to_cluster(risk_category):
    if risk_category == 'Low':
        return 0  # Low-risk cluster
    elif risk_category == 'Medium':
        return 1  # Medium-risk cluster
    else:
        return 2  # High-risk cluster

# Example new genetic profile
new_profile = pd.DataFrame({
    'BMI': [24.5],
    'MC4R_Variant': ['rs17782313_TT'],
    'PPARG_Variant': ['rs1801282_CC'],
    'FTO_Variant': ['rs9939609_TT'],
    'LEPR_Variant': ['rs1137101_AG'],
    'Physical_Activity': ['High'],
    'Diet_Type': ['High-Fat']
})

# Encode categorical variables in the new profile
new_profile = pd.get_dummies(new_profile, drop_first=True)

# Ensure the new profile has the same columns as the training data
missing_cols = set(X_train.columns) - set(new_profile.columns)
for col in missing_cols:
    new_profile[col] = 0
new_profile = new_profile[X_train.columns]

# Predict risk category for the new profile
predicted_risk_category = best_rf.predict(new_profile)
print(f'Predicted Obesity Risk Category: {predicted_risk_category[0]}')

# Map predicted risk category to a meal cluster
recommended_cluster = map_risk_to_cluster(predicted_risk_category[0])
print(f'Recommended Meal Cluster: {recommended_cluster}')

# Recommend meals from the recommended cluster
recommended_meals = meal_df[meal_df['Meal_Cluster'] == recommended_cluster]
print(recommended_meals[['Descrip', 'Energy_kcal', 'Protein_g', 'Fat_g', 'Carb_g']])

Best Parameters: {'max_depth': 10, 'min_samples_split': 10, 'n_estimators': 100}
Accuracy: 0.4175
Precision: 0.35652521929824565
Recall: 0.4175
F1 Score: 0.37008044889766223
Predicted Obesity Risk Category: Medium
Recommended Meal Cluster: 1
                                                Descrip  Energy_kcal  \
0                          Soy flour, full-fat, roasted        441.0   
15    Fast foods, english muffin, with cheese and sa...        338.0   
19         Peanuts, virginia, oil-roasted, without salt        578.0   
20              Cheesefurter, cheese smokie, pork, beef        328.0   
26    Restaurant, family style, fried mozzarella sticks        325.0   
...                                                 ...          ...   
6869  BURGER KING, CROISSAN'WICH with Sausage and Ch...        376.0   
6874  Pork, fresh, spareribs, separable lean and fat...        397.0   
6877  Margarine-like, vegetable oil-butter spread, r...        450.0   
6879  DOMINO'S 14" Pepperoni Pizza, Cl

In [13]:
print(y_genetic.value_counts())

Obesity_Risk_Category
High      848
Medium    696
Low       456
Name: count, dtype: int64


In [15]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.cluster import KMeans
from imblearn.over_sampling import SMOTE  # Import SMOTE

# Load datasets
genetic_df = pd.read_csv("C:\\Users\\trejan\\Desktop\\Sem 2\\Machine Learning\\model\\genetic_profiles.csv")
meal_df = pd.read_csv("C:\\Users\\trejan\\Desktop\\Sem 2\\Machine Learning\\model\\train.csv")

# Handle missing values for numeric columns in genetic_df
numeric_cols = genetic_df.select_dtypes(include=['float64', 'int64']).columns
genetic_df[numeric_cols] = genetic_df[numeric_cols].fillna(genetic_df[numeric_cols].mean())

# Handle missing values for categorical columns in genetic_df
categorical_cols = genetic_df.select_dtypes(include=['object']).columns
genetic_df[categorical_cols] = genetic_df[categorical_cols].fillna('Unknown')

# Encode categorical columns in genetic_df
le = LabelEncoder()
for col in categorical_cols:
    genetic_df[col] = le.fit_transform(genetic_df[col])

# Convert Obesity_Risk_Score into categories (low, medium, high)
genetic_df['Obesity_Risk_Category'] = pd.cut(
    genetic_df['Obesity_Risk_Score'],
    bins=[0, 0.3, 0.6, 1],
    labels=['Low', 'Medium', 'High']
)

# Prepare features and target for genetic data
X_genetic = genetic_df.drop(['Obesity_Risk_Score', 'Obesity_Risk_Category'], axis=1)
y_genetic = genetic_df['Obesity_Risk_Category']

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_genetic, y_genetic)

# Split resampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Hyperparameter tuning for Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

# Use RandomForestClassifier for classification
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(f'Best Parameters: {best_params}')

# Train the model with the best parameters
best_rf = grid_search.best_estimator_
best_rf.fit(X_train, y_train)

# Evaluate the model
y_pred = best_rf.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred) * 100
precision = precision_score(y_test, y_pred, average='weighted') 
recall = recall_score(y_test, y_pred, average='weighted') 
f1 = f1_score(y_test, y_pred, average='weighted') 

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

# Print classification report and confusion matrix
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Preprocess nutritional data
nutritional_features = meal_df[['Energy_kcal', 'Protein_g', 'Fat_g', 'Carb_g']]
scaler = StandardScaler()
nutritional_features_scaled = scaler.fit_transform(nutritional_features)

# Cluster meals using K-Means
kmeans = KMeans(n_clusters=5, random_state=42)
meal_df['Meal_Cluster'] = kmeans.fit_predict(nutritional_features_scaled)

# Define a mapping function to map risk categories to meal clusters
def map_risk_to_cluster(risk_category):
    if risk_category == 'Low':
        return 0  # Low-risk cluster
    elif risk_category == 'Medium':
        return 1  # Medium-risk cluster
    else:
        return 2  # High-risk cluster

# Example new genetic profile
new_profile = pd.DataFrame({
    'BMI': [24.5],
    'MC4R_Variant': ['rs17782313_TT'],
    'PPARG_Variant': ['rs1801282_CC'],
    'FTO_Variant': ['rs9939609_TT'],
    'LEPR_Variant': ['rs1137101_AG'],
    'Physical_Activity': ['High'],
    'Diet_Type': ['High-Fat']
})

# Encode categorical variables in the new profile
new_profile = pd.get_dummies(new_profile, drop_first=True)

# Ensure the new profile has the same columns as the training data
missing_cols = set(X_train.columns) - set(new_profile.columns)
for col in missing_cols:
    new_profile[col] = 0
new_profile = new_profile[X_train.columns]

# Predict risk category for the new profile
predicted_risk_category = best_rf.predict(new_profile)
print(f'Predicted Obesity Risk Category: {predicted_risk_category[0]}')

# Map predicted risk category to a meal cluster
recommended_cluster = map_risk_to_cluster(predicted_risk_category[0])
print(f'Recommended Meal Cluster: {recommended_cluster}')

# Recommend meals from the recommended cluster
recommended_meals = meal_df[meal_df['Meal_Cluster'] == recommended_cluster]
print(recommended_meals[['Descrip', 'Energy_kcal', 'Protein_g', 'Fat_g', 'Carb_g']])

Best Parameters: {'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 200}
Accuracy: 48.52652259332024
Precision: 48.332260336442836
Recall: 48.52652259332024
F1 Score: 48.38473848257559

Classification Report:
              precision    recall  f1-score   support

        High       0.50      0.47      0.48       184
         Low       0.56      0.60      0.58       161
      Medium       0.40      0.39      0.39       164

    accuracy                           0.49       509
   macro avg       0.48      0.49      0.48       509
weighted avg       0.48      0.49      0.48       509


Confusion Matrix:
[[86 34 64]
 [30 97 34]
 [57 43 64]]
Predicted Obesity Risk Category: Medium
Recommended Meal Cluster: 1
                                                Descrip  Energy_kcal  \
0                          Soy flour, full-fat, roasted        441.0   
15    Fast foods, english muffin, with cheese and sa...        338.0   
19         Peanuts, virginia, oil-roasted, without salt        

In [16]:
importances = best_rf.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': importances})
print(feature_importance_df.sort_values(by='Importance', ascending=False))

             Feature  Importance
0         Profile_ID    0.243472
2                BMI    0.224138
1                Age    0.203519
8          Diet_Type    0.057196
5        FTO_Variant    0.056020
3       MC4R_Variant    0.055008
4      PPARG_Variant    0.054802
6       LEPR_Variant    0.053823
7  Physical_Activity    0.052023


In [17]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.cluster import KMeans
from imblearn.over_sampling import SMOTE

# Load datasets
genetic_df = pd.read_csv("C:\\Users\\trejan\\Desktop\\Sem 2\\Machine Learning\\model\\genetic_profiles.csv")
meal_df = pd.read_csv("C:\\Users\\trejan\\Desktop\\Sem 2\\Machine Learning\\model\\train.csv")

# Handle missing values for numeric columns in genetic_df
numeric_cols = genetic_df.select_dtypes(include=['float64', 'int64']).columns
genetic_df[numeric_cols] = genetic_df[numeric_cols].fillna(genetic_df[numeric_cols].mean())

# Handle missing values for categorical columns in genetic_df
categorical_cols = genetic_df.select_dtypes(include=['object']).columns
genetic_df[categorical_cols] = genetic_df[categorical_cols].fillna('Unknown')

# Convert Obesity_Risk_Score into categories (low, medium, high)
genetic_df['Obesity_Risk_Category'] = pd.cut(
    genetic_df['Obesity_Risk_Score'],
    bins=[0, 0.3, 0.6, 1],
    labels=['Low', 'Medium', 'High']
)

# Drop irrelevant features (Profile_ID, Age, Physical_Activity)
genetic_df = genetic_df.drop(['Profile_ID', 'Age', 'Physical_Activity'], axis=1)

# One-hot encode categorical features (Diet_Type and genetic variants)
genetic_df = pd.get_dummies(genetic_df, columns=['Diet_Type', 'MC4R_Variant', 'PPARG_Variant', 'FTO_Variant', 'LEPR_Variant'], drop_first=True)

# Prepare features and target
X_genetic = genetic_df.drop(['Obesity_Risk_Score', 'Obesity_Risk_Category'], axis=1)
y_genetic = genetic_df['Obesity_Risk_Category']

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_genetic, y_genetic)

# Split resampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Hyperparameter tuning for Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

# Use RandomForestClassifier for classification
rf = RandomForestClassifier(random_state=42, class_weight='balanced')
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(f'Best Parameters: {best_params}')

# Train the model with the best parameters
best_rf = grid_search.best_estimator_
best_rf.fit(X_train, y_train)

# Evaluate the model
y_pred = best_rf.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

# Print classification report and confusion matrix
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Preprocess nutritional data
nutritional_features = meal_df[['Energy_kcal', 'Protein_g', 'Fat_g', 'Carb_g']]
scaler = StandardScaler()
nutritional_features_scaled = scaler.fit_transform(nutritional_features)

# Cluster meals using K-Means
kmeans = KMeans(n_clusters=5, random_state=42)
meal_df['Meal_Cluster'] = kmeans.fit_predict(nutritional_features_scaled)

# Define a mapping function to map risk categories to meal clusters
def map_risk_to_cluster(risk_category):
    if risk_category == 'Low':
        return 0  # Low-risk cluster
    elif risk_category == 'Medium':
        return 1  # Medium-risk cluster
    else:
        return 2  # High-risk cluster

# Example new genetic profile
new_profile = pd.DataFrame({
    'BMI': [24.5],
    'MC4R_Variant_rs17782313_CT': [0],
    'MC4R_Variant_rs17782313_TT': [1],
    'PPARG_Variant_rs1801282_CG': [0],
    'PPARG_Variant_rs1801282_GG': [0],
    'FTO_Variant_rs9939609_AT': [0],
    'FTO_Variant_rs9939609_TT': [1],
    'LEPR_Variant_rs1137101_AG': [1],
    'LEPR_Variant_rs1137101_GG': [0],
    'Diet_Type_High-Carb': [0],
    'Diet_Type_High-Fat': [1]
})

# Ensure the new profile has the same columns as the training data
missing_cols = set(X_train.columns) - set(new_profile.columns)
for col in missing_cols:
    new_profile[col] = 0
new_profile = new_profile[X_train.columns]

# Predict risk category for the new profile
predicted_risk_category = best_rf.predict(new_profile)
print(f'Predicted Obesity Risk Category: {predicted_risk_category[0]}')

# Map predicted risk category to a meal cluster
recommended_cluster = map_risk_to_cluster(predicted_risk_category[0])
print(f'Recommended Meal Cluster: {recommended_cluster}')

# Recommend meals from the recommended cluster
recommended_meals = meal_df[meal_df['Meal_Cluster'] == recommended_cluster]
print(recommended_meals[['Descrip', 'Energy_kcal', 'Protein_g', 'Fat_g', 'Carb_g']])

Best Parameters: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 300}
Accuracy: 0.4911591355599214
Precision: 0.4885500967102819
Recall: 0.4911591355599214
F1 Score: 0.48830630696733346

Classification Report:
              precision    recall  f1-score   support

        High       0.51      0.45      0.48       184
         Low       0.55      0.63      0.59       161
      Medium       0.40      0.40      0.40       164

    accuracy                           0.49       509
   macro avg       0.49      0.49      0.49       509
weighted avg       0.49      0.49      0.49       509


Confusion Matrix:
[[ 83  43  58]
 [ 21 102  38]
 [ 59  40  65]]
Predicted Obesity Risk Category: Medium
Recommended Meal Cluster: 1
                                                Descrip  Energy_kcal  \
0                          Soy flour, full-fat, roasted        441.0   
15    Fast foods, english muffin, with cheese and sa...        338.0   
19         Peanuts, virginia, oil-roasted, without

In [19]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.cluster import KMeans
from imblearn.over_sampling import SMOTE

# Load datasets
genetic_df = pd.read_csv("C:\\Users\\trejan\\Desktop\\Sem 2\\Machine Learning\\model\\genetic_profiles.csv")
meal_df = pd.read_csv("C:\\Users\\trejan\\Desktop\\Sem 2\\Machine Learning\\model\\train.csv")

# Handle missing values for numeric columns in genetic_df
numeric_cols = genetic_df.select_dtypes(include=['float64', 'int64']).columns
genetic_df[numeric_cols] = genetic_df[numeric_cols].fillna(genetic_df[numeric_cols].mean())

# Handle missing values for categorical columns in genetic_df
categorical_cols = genetic_df.select_dtypes(include=['object']).columns
genetic_df[categorical_cols] = genetic_df[categorical_cols].fillna('Unknown')

# Convert Obesity_Risk_Score into categories (low, medium, high)
genetic_df['Obesity_Risk_Category'] = pd.cut(
    genetic_df['Obesity_Risk_Score'],
    bins=[0, 0.3, 0.6, 1],
    labels=['Low', 'Medium', 'High']
)

# Select relevant features
features = ['BMI', 'Physical_Activity', 'Diet_Type', 'MC4R_Variant', 'PPARG_Variant', 'FTO_Variant', 'LEPR_Variant', 'Obesity_Risk_Score']
genetic_df = genetic_df[features + ['Obesity_Risk_Category']]

# One-hot encode categorical features (Diet_Type and genetic variants)
genetic_df = pd.get_dummies(genetic_df, columns=['Diet_Type', 'MC4R_Variant', 'PPARG_Variant', 'FTO_Variant', 'LEPR_Variant'], drop_first=True)

# Prepare features and target
X = genetic_df.drop(['Obesity_Risk_Category'], axis=1)
y = genetic_df['Obesity_Risk_Category']

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Split resampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Hyperparameter tuning for Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Use RandomForestClassifier for classification
rf = RandomForestClassifier(random_state=42, class_weight='balanced')
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='f1_weighted')
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(f'Best Parameters: {best_params}')

# Train the model with the best parameters
best_rf = grid_search.best_estimator_
best_rf.fit(X_train, y_train)

# Evaluate the model
y_pred = best_rf.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

# Print classification report and confusion matrix
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Preprocess nutritional data for meal recommendations
nutritional_features = meal_df[['Energy_kcal', 'Protein_g', 'Fat_g', 'Carb_g']]
scaler = StandardScaler()
nutritional_features_scaled = scaler.fit_transform(nutritional_features)

# Cluster meals using K-Means
kmeans = KMeans(n_clusters=5, random_state=42)
meal_df['Meal_Cluster'] = kmeans.fit_predict(nutritional_features_scaled)

# Define a mapping function to map risk categories to meal clusters
def map_risk_to_cluster(risk_category):
    if risk_category == 'Low':
        return 0  # Low-risk cluster
    elif risk_category == 'Medium':
        return 1  # Medium-risk cluster
    else:
        return 2  # High-risk cluster

# Example new genetic profile
new_profile = pd.DataFrame({
    'BMI': [24.5],
    'Physical_Activity': [3],  # Assuming 3 is moderate activity
    'Diet_Type_High-Carb': [0],
    'Diet_Type_High-Fat': [1],
    'MC4R_Variant_rs17782313_CT': [0],
    'MC4R_Variant_rs17782313_TT': [1],
    'PPARG_Variant_rs1801282_CG': [0],
    'PPARG_Variant_rs1801282_GG': [0],
    'FTO_Variant_rs9939609_AT': [0],
    'FTO_Variant_rs9939609_TT': [1],
    'LEPR_Variant_rs1137101_AG': [1],
    'LEPR_Variant_rs1137101_GG': [0],
    'Obesity_Risk_Score': [0.55]
})

# Ensure the new profile has the same columns as the training data
missing_cols = set(X_train.columns) - set(new_profile.columns)
for col in missing_cols:
    new_profile[col] = 0
new_profile = new_profile[X_train.columns]

# Predict risk category for the new profile
predicted_risk_category = best_rf.predict(new_profile)
print(f'Predicted Obesity Risk Category: {predicted_risk_category[0]}')

# Map predicted risk category to a meal cluster
recommended_cluster = map_risk_to_cluster(predicted_risk_category[0])
print(f'Recommended Meal Cluster: {recommended_cluster}')

# Recommend meals from the recommended cluster
recommended_meals = meal_df[meal_df['Meal_Cluster'] == recommended_cluster]
print(recommended_meals[['Descrip', 'Energy_kcal', 'Protein_g', 'Fat_g', 'Carb_g']])

ValueError: could not convert string to float: 'High'

In [23]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.cluster import KMeans
from imblearn.over_sampling import SMOTE

# Load datasets
genetic_df = pd.read_csv("C:\\Users\\trejan\\Desktop\\Sem 2\\Machine Learning\\model\\genetic_profiles.csv")
meal_df = pd.read_csv("C:\\Users\\trejan\\Desktop\\Sem 2\\Machine Learning\\model\\train.csv")

# Handle missing values for numeric columns in genetic_df
numeric_cols = genetic_df.select_dtypes(include=['float64', 'int64']).columns
genetic_df[numeric_cols] = genetic_df[numeric_cols].fillna(genetic_df[numeric_cols].mean())

# Handle missing values for categorical columns in genetic_df
categorical_cols = genetic_df.select_dtypes(include=['object']).columns
genetic_df[categorical_cols] = genetic_df[categorical_cols].fillna('Unknown')

# Convert Obesity_Risk_Score into categories (low, medium, high)
genetic_df['Obesity_Risk_Category'] = pd.cut(
    genetic_df['Obesity_Risk_Score'],
    bins=[0, 0.3, 0.6, 1],
    labels=['Low', 'Medium', 'High']
)

# Select relevant features
features = ['BMI', 'Physical_Activity', 'Diet_Type', 'MC4R_Variant', 'PPARG_Variant', 'FTO_Variant', 'LEPR_Variant', 'Obesity_Risk_Score']
genetic_df = genetic_df[features + ['Obesity_Risk_Category']]

# One-hot encode categorical features (Diet_Type and genetic variants)
genetic_df = pd.get_dummies(genetic_df, columns=['Diet_Type', 'MC4R_Variant', 'PPARG_Variant', 'FTO_Variant', 'LEPR_Variant'], drop_first=True)

# Prepare features and target
X = genetic_df.drop(['Obesity_Risk_Category'], axis=1)
y = genetic_df['Obesity_Risk_Category']

# Convert Obesity_Risk_Category to numeric labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Ensure all columns in X are numeric
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = pd.to_numeric(X[col], errors='coerce')  # Convert to numeric, coercing errors to NaN
        X[col] = X[col].fillna(0)  # Fill NaN values with 0

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y_encoded)

# Split resampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Convert y_train and y_test back to original labels for evaluation
y_train_labels = label_encoder.inverse_transform(y_train)
y_test_labels = label_encoder.inverse_transform(y_test)

# Hyperparameter tuning for Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Use RandomForestClassifier for classification
rf = RandomForestClassifier(random_state=42, class_weight='balanced')
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='f1_weighted')
grid_search.fit(X_train, y_train_labels)

# Get the best parameters
best_params = grid_search.best_params_
print(f'Best Parameters: {best_params}')

# Train the model with the best parameters
best_rf = grid_search.best_estimator_
best_rf.fit(X_train, y_train_labels)

# Evaluate the model
y_pred = best_rf.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test_labels, y_pred)
precision = precision_score(y_test_labels, y_pred, average='weighted')
recall = recall_score(y_test_labels, y_pred, average='weighted')
f1 = f1_score(y_test_labels, y_pred, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

# Print classification report and confusion matrix
print("\nClassification Report:")
print(classification_report(y_test_labels, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test_labels, y_pred))

# Preprocess nutritional data for meal recommendations
nutritional_features = meal_df[['Energy_kcal', 'Protein_g', 'Fat_g', 'Carb_g']]
scaler = StandardScaler()
nutritional_features_scaled = scaler.fit_transform(nutritional_features)

# Cluster meals using K-Means
kmeans = KMeans(n_clusters=5, random_state=42)
meal_df['Meal_Cluster'] = kmeans.fit_predict(nutritional_features_scaled)

# Define a mapping function to map risk categories to meal clusters
def map_risk_to_cluster(risk_category):
    if risk_category == 'Low':
        return 0  # Low-risk cluster
    elif risk_category == 'Medium':
        return 1  # Medium-risk cluster
    else:
        return 2  # High-risk cluster

# Example new genetic profile
new_profile = pd.DataFrame({
    'BMI': [24.5],
    'Physical_Activity': [3],  # Assuming 3 is moderate activity
    'Diet_Type_High-Carb': [0],
    'Diet_Type_High-Fat': [1],
    'MC4R_Variant_rs17782313_CT': [0],
    'MC4R_Variant_rs17782313_TT': [1],
    'PPARG_Variant_rs1801282_CG': [0],
    'PPARG_Variant_rs1801282_GG': [0],
    'FTO_Variant_rs9939609_AT': [0],
    'FTO_Variant_rs9939609_TT': [1],
    'LEPR_Variant_rs1137101_AG': [1],
    'LEPR_Variant_rs1137101_GG': [0],
    'Obesity_Risk_Score': [0.55]
})

# Ensure the new profile has the same columns as the training data
missing_cols = set(X_train.columns) - set(new_profile.columns)
for col in missing_cols:
    new_profile[col] = 0
new_profile = new_profile[X_train.columns]

# Predict risk category for the new profile
predicted_risk_category = best_rf.predict(new_profile)
print(f'Predicted Obesity Risk Category: {predicted_risk_category[0]}')

# Map predicted risk category to a meal cluster
recommended_cluster = map_risk_to_cluster(predicted_risk_category[0])
print(f'Recommended Meal Cluster: {recommended_cluster}')

# Recommend meals from the recommended cluster
recommended_meals = meal_df[meal_df['Meal_Cluster'] == recommended_cluster]
print(recommended_meals[['Descrip', 'Energy_kcal', 'Protein_g', 'Fat_g', 'Carb_g']])

C:\Users\trejan\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
405 fits failed out of a total of 1215.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
405 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\trejan\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\trejan\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File

Best Parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

Classification Report:
              precision    recall  f1-score   support

        High       1.00      1.00      1.00       184
         Low       1.00      1.00      1.00       161
      Medium       1.00      1.00      1.00       164

    accuracy                           1.00       509
   macro avg       1.00      1.00      1.00       509
weighted avg       1.00      1.00      1.00       509


Confusion Matrix:
[[184   0   0]
 [  0 161   0]
 [  0   0 164]]
Predicted Obesity Risk Category: Medium
Recommended Meal Cluster: 1
                                                Descrip  Energy_kcal  \
0                          Soy flour, full-fat, roasted        441.0   
15    Fast foods, english muffin, with cheese and sa...        338.0   
19         Peanuts, virginia, oil-roasted, without salt       

In [24]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.cluster import KMeans
from imblearn.over_sampling import SMOTE

# Load datasets
genetic_df = pd.read_csv("C:\\Users\\trejan\\Desktop\\Sem 2\\Machine Learning\\model\\genetic_profiles.csv")
meal_df = pd.read_csv("C:\\Users\\trejan\\Desktop\\Sem 2\\Machine Learning\\model\\train.csv")

# Handle missing values for numeric columns in genetic_df
numeric_cols = genetic_df.select_dtypes(include=['float64', 'int64']).columns
genetic_df[numeric_cols] = genetic_df[numeric_cols].fillna(genetic_df[numeric_cols].mean())

# Handle missing values for categorical columns in genetic_df
categorical_cols = genetic_df.select_dtypes(include=['object']).columns
genetic_df[categorical_cols] = genetic_df[categorical_cols].fillna('Unknown')

# Convert Obesity_Risk_Score into categories (low, medium, high)
genetic_df['Obesity_Risk_Category'] = pd.cut(
    genetic_df['Obesity_Risk_Score'],
    bins=[0, 0.3, 0.6, 1],
    labels=['Low', 'Medium', 'High']
)

# Select relevant features
features = ['BMI', 'Physical_Activity', 'Diet_Type', 'MC4R_Variant', 'PPARG_Variant', 'FTO_Variant', 'LEPR_Variant', 'Obesity_Risk_Score']
genetic_df = genetic_df[features + ['Obesity_Risk_Category']]

# One-hot encode categorical features (Diet_Type and genetic variants)
genetic_df = pd.get_dummies(genetic_df, columns=['Diet_Type', 'MC4R_Variant', 'PPARG_Variant', 'FTO_Variant', 'LEPR_Variant'], drop_first=True)

# Prepare features and target
X = genetic_df.drop(['Obesity_Risk_Category'], axis=1)
y = genetic_df['Obesity_Risk_Category']

# Convert Obesity_Risk_Category to numeric labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Ensure all columns in X are numeric
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = pd.to_numeric(X[col], errors='coerce')  # Convert to numeric, coercing errors to NaN
        X[col] = X[col].fillna(0)  # Fill NaN values with 0

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y_encoded)

# Split resampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Convert y_train and y_test back to original labels for evaluation
y_train_labels = label_encoder.inverse_transform(y_train)
y_test_labels = label_encoder.inverse_transform(y_test)

# Updated Hyperparameter Grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']  # Removed 'auto'
}

# Use RandomForestClassifier for classification
rf = RandomForestClassifier(random_state=42, class_weight='balanced')
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='f1_weighted')
grid_search.fit(X_train, y_train_labels)

# Get the best parameters
best_params = grid_search.best_params_
print(f'Best Parameters: {best_params}')

# Train the model with the best parameters
best_rf = grid_search.best_estimator_
best_rf.fit(X_train, y_train_labels)

# Evaluate the model
y_pred = best_rf.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test_labels, y_pred)
precision = precision_score(y_test_labels, y_pred, average='weighted')
recall = recall_score(y_test_labels, y_pred, average='weighted')
f1 = f1_score(y_test_labels, y_pred, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

# Print classification report and confusion matrix
print("\nClassification Report:")
print(classification_report(y_test_labels, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test_labels, y_pred))

# Preprocess nutritional data for meal recommendations
nutritional_features = meal_df[['Energy_kcal', 'Protein_g', 'Fat_g', 'Carb_g']]
scaler = StandardScaler()
nutritional_features_scaled = scaler.fit_transform(nutritional_features)

# Cluster meals using K-Means
kmeans = KMeans(n_clusters=5, random_state=42)
meal_df['Meal_Cluster'] = kmeans.fit_predict(nutritional_features_scaled)

# Define a mapping function to map risk categories to meal clusters
def map_risk_to_cluster(risk_category):
    if risk_category == 'Low':
        return 0  # Low-risk cluster
    elif risk_category == 'Medium':
        return 1  # Medium-risk cluster
    else:
        return 2  # High-risk cluster

# Example new genetic profile
new_profile = pd.DataFrame({
    'BMI': [24.5],
    'Physical_Activity': [3],  # Assuming 3 is moderate activity
    'Diet_Type_High-Carb': [0],
    'Diet_Type_High-Fat': [1],
    'MC4R_Variant_rs17782313_CT': [0],
    'MC4R_Variant_rs17782313_TT': [1],
    'PPARG_Variant_rs1801282_CG': [0],
    'PPARG_Variant_rs1801282_GG': [0],
    'FTO_Variant_rs9939609_AT': [0],
    'FTO_Variant_rs9939609_TT': [1],
    'LEPR_Variant_rs1137101_AG': [1],
    'LEPR_Variant_rs1137101_GG': [0],
    'Obesity_Risk_Score': [0.55]
})

# Ensure the new profile has the same columns as the training data
missing_cols = set(X_train.columns) - set(new_profile.columns)
for col in missing_cols:
    new_profile[col] = 0
new_profile = new_profile[X_train.columns]

# Predict risk category for the new profile
predicted_risk_category = best_rf.predict(new_profile)
print(f'Predicted Obesity Risk Category: {predicted_risk_category[0]}')

# Map predicted risk category to a meal cluster
recommended_cluster = map_risk_to_cluster(predicted_risk_category[0])
print(f'Recommended Meal Cluster: {recommended_cluster}')

# Recommend meals from the recommended cluster
recommended_meals = meal_df[meal_df['Meal_Cluster'] == recommended_cluster]
print(recommended_meals[['Descrip', 'Energy_kcal', 'Protein_g', 'Fat_g', 'Carb_g']])

Best Parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

Classification Report:
              precision    recall  f1-score   support

        High       1.00      1.00      1.00       184
         Low       1.00      1.00      1.00       161
      Medium       1.00      1.00      1.00       164

    accuracy                           1.00       509
   macro avg       1.00      1.00      1.00       509
weighted avg       1.00      1.00      1.00       509


Confusion Matrix:
[[184   0   0]
 [  0 161   0]
 [  0   0 164]]
Predicted Obesity Risk Category: Medium
Recommended Meal Cluster: 1
                                                Descrip  Energy_kcal  \
0                          Soy flour, full-fat, roasted        441.0   
15    Fast foods, english muffin, with cheese and sa...        338.0   
19         Peanuts, virginia, oil-roasted, without salt       

In [25]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, precision_recall_fscore_support
from sklearn.cluster import KMeans
from imblearn.over_sampling import SMOTE

# Load datasets
genetic_df = pd.read_csv("C:\\Users\\trejan\\Desktop\\Sem 2\\Machine Learning\\model\\genetic_profiles.csv")
meal_df = pd.read_csv("C:\\Users\\trejan\\Desktop\\Sem 2\\Machine Learning\\model\\train.csv")

# Handle missing values for numeric columns in genetic_df
numeric_cols = genetic_df.select_dtypes(include=['float64', 'int64']).columns
genetic_df[numeric_cols] = genetic_df[numeric_cols].fillna(genetic_df[numeric_cols].mean())

# Handle missing values for categorical columns in genetic_df
categorical_cols = genetic_df.select_dtypes(include=['object']).columns
genetic_df[categorical_cols] = genetic_df[categorical_cols].fillna('Unknown')

# Convert Obesity_Risk_Score into categories (low, medium, high)
genetic_df['Obesity_Risk_Category'] = pd.cut(
    genetic_df['Obesity_Risk_Score'],
    bins=[0, 0.3, 0.6, 1],
    labels=['Low', 'Medium', 'High']
)

# Select relevant features
features = ['BMI', 'Physical_Activity', 'Diet_Type', 'MC4R_Variant', 'PPARG_Variant', 'FTO_Variant', 'LEPR_Variant', 'Obesity_Risk_Score']
genetic_df = genetic_df[features + ['Obesity_Risk_Category']]

# One-hot encode categorical features (Diet_Type and genetic variants)
genetic_df = pd.get_dummies(genetic_df, columns=['Diet_Type', 'MC4R_Variant', 'PPARG_Variant', 'FTO_Variant', 'LEPR_Variant'], drop_first=True)

# Prepare features and target
X = genetic_df.drop(['Obesity_Risk_Category'], axis=1)
y = genetic_df['Obesity_Risk_Category']

# Convert Obesity_Risk_Category to numeric labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Ensure all columns in X are numeric
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = pd.to_numeric(X[col], errors='coerce')  # Convert to numeric, coercing errors to NaN
        X[col] = X[col].fillna(0)  # Fill NaN values with 0

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y_encoded)

# Split resampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Convert y_train and y_test back to original labels for evaluation
y_train_labels = label_encoder.inverse_transform(y_train)
y_test_labels = label_encoder.inverse_transform(y_test)

# Updated Hyperparameter Grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']  # Removed 'auto'
}

# Use RandomForestClassifier for classification
rf = RandomForestClassifier(random_state=42, class_weight='balanced')
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='f1_weighted')
grid_search.fit(X_train, y_train_labels)

# Get the best parameters
best_params = grid_search.best_params_
print(f'Best Parameters: {best_params}')

# Train the model with the best parameters
best_rf = grid_search.best_estimator_
best_rf.fit(X_train, y_train_labels)

# Evaluate the model
y_pred = best_rf.predict(X_test)

# Calculate metrics as percentages
accuracy = accuracy_score(y_test_labels, y_pred) * 100
precision = precision_score(y_test_labels, y_pred, average='weighted') * 100
recall = recall_score(y_test_labels, y_pred, average='weighted') * 100
f1 = f1_score(y_test_labels, y_pred, average='weighted') * 100

# Print metrics as percentages
print(f'Accuracy: {accuracy:.2f}%')
print(f'Precision: {precision:.2f}%')
print(f'Recall: {recall:.2f}%')
print(f'F1 Score: {f1:.2f}%')

# Calculate precision, recall, and F1 score for each class
precision, recall, f1, _ = precision_recall_fscore_support(y_test_labels, y_pred, average=None)

# Convert to percentages
precision = precision * 100
recall = recall * 100
f1 = f1 * 100

# Print formatted classification report
print("\nClassification Report (Percentage):")
print(f"{'Class':<10} {'Precision':<10} {'Recall':<10} {'F1 Score':<10}")
for i, class_name in enumerate(label_encoder.classes_):
    print(f"{class_name:<10} {precision[i]:<10.2f}% {recall[i]:<10.2f}% {f1[i]:<10.2f}%")

# Print confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test_labels, y_pred))

# Preprocess nutritional data for meal recommendations
nutritional_features = meal_df[['Energy_kcal', 'Protein_g', 'Fat_g', 'Carb_g']]
scaler = StandardScaler()
nutritional_features_scaled = scaler.fit_transform(nutritional_features)

# Cluster meals using K-Means
kmeans = KMeans(n_clusters=5, random_state=42)
meal_df['Meal_Cluster'] = kmeans.fit_predict(nutritional_features_scaled)

# Define a mapping function to map risk categories to meal clusters
def map_risk_to_cluster(risk_category):
    if risk_category == 'Low':
        return 0  # Low-risk cluster
    elif risk_category == 'Medium':
        return 1  # Medium-risk cluster
    else:
        return 2  # High-risk cluster

# Function to display meal clusters in a user-friendly format
def display_meal_cluster(meals, cluster_name, num_meals=5):
    print(f"\n=== Recommended Meals for {cluster_name} Cluster ===")
    print(f"Total Meals in Cluster: {len(meals)}")
    
    # Display a subset of meals
    print(f"\nTop {num_meals} Meals:")
    print(f"{'Meal Description':<50} {'Energy (kcal)':<15} {'Protein (g)':<15} {'Fat (g)':<15} {'Carbs (g)':<15}")
    print("-" * 110)
    for _, meal in meals.head(num_meals).iterrows():
        print(f"{meal['Descrip']:<50} {meal['Energy_kcal']:<15} {meal['Protein_g']:<15} {meal['Fat_g']:<15} {meal['Carb_g']:<15}")
    
    # Display summary of nutritional values
    avg_energy = meals['Energy_kcal'].mean()
    avg_protein = meals['Protein_g'].mean()
    avg_fat = meals['Fat_g'].mean()
    avg_carbs = meals['Carb_g'].mean()
    
    print("\n=== Nutritional Summary ===")
    print(f"Average Energy (kcal): {avg_energy:.2f}")
    print(f"Average Protein (g): {avg_protein:.2f}")
    print(f"Average Fat (g): {avg_fat:.2f}")
    print(f"Average Carbs (g): {avg_carbs:.2f}")

# Map cluster numbers to meaningful names
cluster_names = {
    0: "Low-Risk",
    1: "Medium-Risk",
    2: "High-Risk"
}

# Example new genetic profile
new_profile = pd.DataFrame({
    'BMI': [24.5],
    'Physical_Activity': [3],  # Assuming 3 is moderate activity
    'Diet_Type_High-Carb': [0],
    'Diet_Type_High-Fat': [1],
    'MC4R_Variant_rs17782313_CT': [0],
    'MC4R_Variant_rs17782313_TT': [1],
    'PPARG_Variant_rs1801282_CG': [0],
    'PPARG_Variant_rs1801282_GG': [0],
    'FTO_Variant_rs9939609_AT': [0],
    'FTO_Variant_rs9939609_TT': [1],
    'LEPR_Variant_rs1137101_AG': [1],
    'LEPR_Variant_rs1137101_GG': [0],
    'Obesity_Risk_Score': [0.55]
})

# Ensure the new profile has the same columns as the training data
missing_cols = set(X_train.columns) - set(new_profile.columns)
for col in missing_cols:
    new_profile[col] = 0
new_profile = new_profile[X_train.columns]

# Predict risk category for the new profile
predicted_risk_category = best_rf.predict(new_profile)
print(f'Predicted Obesity Risk Category: {predicted_risk_category[0]}')

# Map predicted risk category to a meal cluster
recommended_cluster = map_risk_to_cluster(predicted_risk_category[0])
recommended_cluster_name = cluster_names.get(recommended_cluster, "Unknown")

# Recommend meals from the recommended cluster
recommended_meals = meal_df[meal_df['Meal_Cluster'] == recommended_cluster]

# Display recommended meals in a user-friendly format
display_meal_cluster(recommended_meals, recommended_cluster_name)

Best Parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Accuracy: 100.00%
Precision: 100.00%
Recall: 100.00%
F1 Score: 100.00%

Classification Report (Percentage):
Class      Precision  Recall     F1 Score  
High       100.00    % 100.00    % 100.00    %
Low        100.00    % 100.00    % 100.00    %
Medium     100.00    % 100.00    % 100.00    %

Confusion Matrix:
[[184   0   0]
 [  0 161   0]
 [  0   0 164]]
Predicted Obesity Risk Category: Medium

=== Recommended Meals for Medium-Risk Cluster ===
Total Meals in Cluster: 779

Top 5 Meals:
Meal Description                                   Energy (kcal)   Protein (g)     Fat (g)         Carbs (g)      
--------------------------------------------------------------------------------------------------------------
Soy flour, full-fat, roasted                       441.0           34.8            21.86           33.67          
Fast foods, english muffin, with cheese

In [27]:
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(best_rf, X_resampled, y_resampled, cv=5, scoring='accuracy')
print(f'Cross-Validation Accuracy: {cv_scores.mean() * 100:.2f}%')

Cross-Validation Accuracy: 100.00%


In [31]:
# Updated Hyperparameter Grid with Regularization
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],  # Limit tree depth
    'min_samples_split': [10, 20],  # Increase minimum samples to split
    'min_samples_leaf': [2, 4],  # Increase minimum samples per leaf
    'max_features': ['sqrt', 'log2']
}

# Use RandomForestClassifier for classification
rf = RandomForestClassifier(random_state=42, class_weight='balanced')
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(f'Best Parameters: {best_params}')

# Train the model with the best parameters
best_rf = grid_search.best_estimator_
best_rf.fit(X_train, y_train)

# Evaluate on noisy data
X_noisy = X_resampled + np.random.normal(0, 0.1, X_resampled.shape)
cv_scores_noisy = cross_val_score(best_rf, X_noisy, y_resampled, cv=5, scoring='accuracy')
print(f'Cross-Validation Accuracy (Noisy Data): {cv_scores_noisy.mean() * 100:.2f}%')

Best Parameters: {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 100}
Cross-Validation Accuracy (Noisy Data): 82.98%


In [32]:
from xgboost import XGBClassifier

xgb = XGBClassifier(random_state=42)
xgb_scores = cross_val_score(xgb, X_noisy, y_resampled, cv=5, scoring='accuracy')
print(f'XGBoost Cross-Validation Accuracy (Noisy Data): {xgb_scores.mean() * 100:.2f}%')

XGBoost Cross-Validation Accuracy (Noisy Data): 81.25%


In [33]:
from sklearn.svm import SVC

svm = SVC(kernel='rbf', random_state=42)
svm_scores = cross_val_score(svm, X_noisy, y_resampled, cv=5, scoring='accuracy')
print(f'SVM Cross-Validation Accuracy (Noisy Data): {svm_scores.mean() * 100:.2f}%')

SVM Cross-Validation Accuracy (Noisy Data): 41.55%


In [34]:
import optuna

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2'])
    }
    model = RandomForestClassifier(**params, random_state=42)
    return cross_val_score(model, X_noisy, y_resampled, cv=5, scoring='accuracy').mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)
print(f'Best Hyperparameters: {study.best_params}')

ModuleNotFoundError: No module named 'optuna'

In [35]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.cluster import KMeans
from imblearn.over_sampling import SMOTE

# Load datasets
genetic_df = pd.read_csv("C:\\Users\\trejan\\Desktop\\Sem 2\\Machine Learning\\model\\genetic_profiles.csv")
meal_df = pd.read_csv("C:\\Users\\trejan\\Desktop\\Sem 2\\Machine Learning\\model\\train.csv")

# Handle missing values for numeric columns in genetic_df
numeric_cols = genetic_df.select_dtypes(include=['float64', 'int64']).columns
genetic_df[numeric_cols] = genetic_df[numeric_cols].fillna(genetic_df[numeric_cols].mean())

# Handle missing values for categorical columns in genetic_df
categorical_cols = genetic_df.select_dtypes(include=['object']).columns
genetic_df[categorical_cols] = genetic_df[categorical_cols].fillna('Unknown')

# Convert Obesity_Risk_Score into categories (low, medium, high)
genetic_df['Obesity_Risk_Category'] = pd.cut(
    genetic_df['Obesity_Risk_Score'],
    bins=[0, 0.3, 0.6, 1],
    labels=['Low', 'Medium', 'High']
)

# Select relevant features
features = ['BMI', 'Physical_Activity', 'Diet_Type', 'MC4R_Variant', 'PPARG_Variant', 'FTO_Variant', 'LEPR_Variant', 'Obesity_Risk_Score']
genetic_df = genetic_df[features + ['Obesity_Risk_Category']]

# One-hot encode categorical features (Diet_Type and genetic variants)
genetic_df = pd.get_dummies(genetic_df, columns=['Diet_Type', 'MC4R_Variant', 'PPARG_Variant', 'FTO_Variant', 'LEPR_Variant'], drop_first=True)

# Prepare features and target
X = genetic_df.drop(['Obesity_Risk_Category'], axis=1)
y = genetic_df['Obesity_Risk_Category']

# Convert Obesity_Risk_Category to numeric labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Ensure all columns in X are numeric
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = pd.to_numeric(X[col], errors='coerce')  # Convert to numeric, coercing errors to NaN
        X[col] = X[col].fillna(0)  # Fill NaN values with 0

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y_encoded)

# Split resampled data into training (60%), validation (20%), and test (20%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X_resampled, y_resampled, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Hyperparameter tuning with GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],  # Limit tree depth
    'min_samples_split': [10, 20],  # Increase minimum samples to split
    'min_samples_leaf': [2, 4],  # Increase minimum samples per leaf
    'max_features': ['sqrt', 'log2']
}

# Use RandomForestClassifier for classification
rf = RandomForestClassifier(random_state=42, class_weight='balanced')
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(f'Best Parameters: {best_params}')

# Train the model with the best parameters
best_rf = grid_search.best_estimator_
best_rf.fit(X_train, y_train)

# Evaluate on the validation set
y_val_pred = best_rf.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred) * 100
print(f'Validation Accuracy: {val_accuracy:.2f}%')

# Evaluate on the test set
y_test_pred = best_rf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred) * 100
print(f'Test Accuracy: {test_accuracy:.2f}%')

# Evaluate on noisy data
X_noisy = X_resampled + np.random.normal(0, 0.1, X_resampled.shape)  # Add Gaussian noise
cv_scores_noisy = cross_val_score(best_rf, X_noisy, y_resampled, cv=5, scoring='accuracy')
print(f'Cross-Validation Accuracy (Noisy Data): {cv_scores_noisy.mean() * 100:.2f}%')

# Print classification report and confusion matrix
print("\nClassification Report (Test Set):")
print(classification_report(y_test, y_test_pred, target_names=label_encoder.classes_))

print("\nConfusion Matrix (Test Set):")
print(confusion_matrix(y_test, y_test_pred))

# Preprocess nutritional data for meal recommendations
nutritional_features = meal_df[['Energy_kcal', 'Protein_g', 'Fat_g', 'Carb_g']]
scaler = StandardScaler()
nutritional_features_scaled = scaler.fit_transform(nutritional_features)

# Cluster meals using K-Means
kmeans = KMeans(n_clusters=5, random_state=42)
meal_df['Meal_Cluster'] = kmeans.fit_predict(nutritional_features_scaled)

# Define a mapping function to map risk categories to meal clusters
def map_risk_to_cluster(risk_category):
    if risk_category == 'Low':
        return 0  # Low-risk cluster
    elif risk_category == 'Medium':
        return 1  # Medium-risk cluster
    else:
        return 2  # High-risk cluster

# Function to display meal clusters in a user-friendly format
def display_meal_cluster(meals, cluster_name, num_meals=5):
    print(f"\n=== Recommended Meals for {cluster_name} Cluster ===")
    print(f"Total Meals in Cluster: {len(meals)}")
    
    # Display a subset of meals
    print(f"\nTop {num_meals} Meals:")
    print(f"{'Meal Description':<50} {'Energy (kcal)':<15} {'Protein (g)':<15} {'Fat (g)':<15} {'Carbs (g)':<15}")
    print("-" * 110)
    for _, meal in meals.head(num_meals).iterrows():
        print(f"{meal['Descrip']:<50} {meal['Energy_kcal']:<15} {meal['Protein_g']:<15} {meal['Fat_g']:<15} {meal['Carb_g']:<15}")
    
    # Display summary of nutritional values
    avg_energy = meals['Energy_kcal'].mean()
    avg_protein = meals['Protein_g'].mean()
    avg_fat = meals['Fat_g'].mean()
    avg_carbs = meals['Carb_g'].mean()
    
    print("\n=== Nutritional Summary ===")
    print(f"Average Energy (kcal): {avg_energy:.2f}")
    print(f"Average Protein (g): {avg_protein:.2f}")
    print(f"Average Fat (g): {avg_fat:.2f}")
    print(f"Average Carbs (g): {avg_carbs:.2f}")

# Map cluster numbers to meaningful names
cluster_names = {
    0: "Low-Risk",
    1: "Medium-Risk",
    2: "High-Risk"
}

# Example new genetic profile
new_profile = {
    'BMI': 28.5,  # Example value
    'Physical_Activity': 2,  # Example value (1: Low, 2: Moderate, 3: High)
    'Diet_Type': 'High-Fat',  # Example value
    'MC4R_Variant': 'rs17782313_CT',  # Example value
    'PPARG_Variant': 'rs1801282_CG',  # Example value
    'FTO_Variant': 'rs9939609_AT',  # Example value
    'LEPR_Variant': 'rs1137101_AG',  # Example value
    'Obesity_Risk_Score': 0.45  # Example value
}

# Convert the new profile to a DataFrame
new_profile_df = pd.DataFrame([new_profile])

# One-hot encode categorical features (same as during training)
new_profile_df = pd.get_dummies(new_profile_df, columns=['Diet_Type', 'MC4R_Variant', 'PPARG_Variant', 'FTO_Variant', 'LEPR_Variant'], drop_first=True)

# Ensure the new profile has the same columns as the training data
missing_cols = set(X_train.columns) - set(new_profile_df.columns)
for col in missing_cols:
    new_profile_df[col] = 0  # Add missing columns with default value 0

# Reorder columns to match the training data
new_profile_df = new_profile_df[X_train.columns]

# Predict the obesity risk category for the new profile
predicted_risk_category = best_rf.predict(new_profile_df)
predicted_risk_category_label = label_encoder.inverse_transform(predicted_risk_category)[0]
print(f'Predicted Obesity Risk Category: {predicted_risk_category_label}')

# Map predicted risk category to a meal cluster
recommended_cluster = map_risk_to_cluster(predicted_risk_category_label)
recommended_cluster_name = cluster_names.get(recommended_cluster, "Unknown")

# Recommend meals from the recommended cluster
recommended_meals = meal_df[meal_df['Meal_Cluster'] == recommended_cluster]

# Display recommended meals in a user-friendly format
display_meal_cluster(recommended_meals, recommended_cluster_name)

Best Parameters: {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 100}
Validation Accuracy: 100.00%
Test Accuracy: 100.00%
Cross-Validation Accuracy (Noisy Data): 82.08%

Classification Report (Test Set):
              precision    recall  f1-score   support

        High       1.00      1.00      1.00       182
         Low       1.00      1.00      1.00       154
      Medium       1.00      1.00      1.00       173

    accuracy                           1.00       509
   macro avg       1.00      1.00      1.00       509
weighted avg       1.00      1.00      1.00       509


Confusion Matrix (Test Set):
[[182   0   0]
 [  0 154   0]
 [  0   0 173]]
Predicted Obesity Risk Category: Medium

=== Recommended Meals for Medium-Risk Cluster ===
Total Meals in Cluster: 779

Top 5 Meals:
Meal Description                                   Energy (kcal)   Protein (g)     Fat (g)         Carbs (g)      
----------------------------------

In [36]:
# Function to recommend meals dynamically based on user profile
def recommend_meals(user_profile, meal_df, model, label_encoder, cluster_names, num_meals=5):
    # Convert the user profile to a DataFrame
    user_profile_df = pd.DataFrame([user_profile])
    
    # One-hot encode categorical features (same as during training)
    user_profile_df = pd.get_dummies(user_profile_df, columns=['Diet_Type', 'MC4R_Variant', 'PPARG_Variant', 'FTO_Variant', 'LEPR_Variant'], drop_first=True)
    
    # Ensure the user profile has the same columns as the training data
    missing_cols = set(X_train.columns) - set(user_profile_df.columns)
    for col in missing_cols:
        user_profile_df[col] = 0  # Add missing columns with default value 0
    
    # Reorder columns to match the training data
    user_profile_df = user_profile_df[X_train.columns]
    
    # Predict the obesity risk category for the user profile
    predicted_risk_category = model.predict(user_profile_df)
    predicted_risk_category_label = label_encoder.inverse_transform(predicted_risk_category)[0]
    print(f'Predicted Obesity Risk Category: {predicted_risk_category_label}')
    
    # Map predicted risk category to multiple clusters (dynamic mapping)
    if predicted_risk_category_label == 'Low':
        clusters = [0, 1]  # Example: Low-risk users get meals from clusters 0 and 1
    elif predicted_risk_category_label == 'Medium':
        clusters = [1, 2]  # Example: Medium-risk users get meals from clusters 1 and 2
    else:
        clusters = [2, 3]  # Example: High-risk users get meals from clusters 2 and 3
    
    # Recommend meals from the selected clusters
    recommended_meals = meal_df[meal_df['Meal_Cluster'].isin(clusters)]
    
    # Sort meals by nutritional suitability (e.g., lower calories for high-risk users)
    if predicted_risk_category_label == 'High':
        recommended_meals = recommended_meals.sort_values(by='Energy_kcal', ascending=True)
    else:
        recommended_meals = recommended_meals.sort_values(by='Protein_g', ascending=False)
    
    # Display recommended meals in a user-friendly format
    display_meal_cluster(recommended_meals, f"{predicted_risk_category_label}-Risk", num_meals)

# Example new genetic profile
new_profile = {
    'BMI': 28.5,  # Example value
    'Physical_Activity': 2,  # Example value (1: Low, 2: Moderate, 3: High)
    'Diet_Type': 'High-Fat',  # Example value
    'MC4R_Variant': 'rs17782313_CT',  # Example value
    'PPARG_Variant': 'rs1801282_CG',  # Example value
    'FTO_Variant': 'rs9939609_AT',  # Example value
    'LEPR_Variant': 'rs1137101_AG',  # Example value
    'Obesity_Risk_Score': 0.45  # Example value
}

# Recommend meals for the new profile
recommend_meals(new_profile, meal_df, best_rf, label_encoder, cluster_names)

Predicted Obesity Risk Category: Medium

=== Recommended Meals for Medium-Risk Cluster ===
Total Meals in Cluster: 950

Top 5 Meals:
Meal Description                                   Energy (kcal)   Protein (g)     Fat (g)         Carbs (g)      
--------------------------------------------------------------------------------------------------------------
Egg, whole, dried                                  605.0           48.37           43.04           1.53           
Egg, whole, dried, stabilized, glucose reduced     615.0           48.17           43.95           2.38           
Tofu, dried-frozen (koyadofu)                      480.0           47.94           30.34           14.56          
Tofu, dried-frozen (koyadofu), prepared with calcium sulfate 472.0           47.94           30.34           12.79          
Cheese, parmesan, low sodium                       451.0           41.6            29.99           3.7            

=== Nutritional Summary ===
Average Energy (kcal): 426.

In [37]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.cluster import KMeans
from imblearn.over_sampling import SMOTE

# Load datasets
genetic_df = pd.read_csv("C:\\Users\\trejan\\Desktop\\Sem 2\\Machine Learning\\model\\genetic_profiles.csv")
meal_df = pd.read_csv("C:\\Users\\trejan\\Desktop\\Sem 2\\Machine Learning\\model\\train.csv")

# Handle missing values for numeric columns in genetic_df
numeric_cols = genetic_df.select_dtypes(include=['float64', 'int64']).columns
genetic_df[numeric_cols] = genetic_df[numeric_cols].fillna(genetic_df[numeric_cols].mean())

# Handle missing values for categorical columns in genetic_df
categorical_cols = genetic_df.select_dtypes(include=['object']).columns
genetic_df[categorical_cols] = genetic_df[categorical_cols].fillna('Unknown')

# Convert Obesity_Risk_Score into categories (low, medium, high)
genetic_df['Obesity_Risk_Category'] = pd.cut(
    genetic_df['Obesity_Risk_Score'],
    bins=[0, 0.3, 0.6, 1],
    labels=['Low', 'Medium', 'High']
)

# Select relevant features
features = ['BMI', 'Physical_Activity', 'Diet_Type', 'MC4R_Variant', 'PPARG_Variant', 'FTO_Variant', 'LEPR_Variant', 'Obesity_Risk_Score']
genetic_df = genetic_df[features + ['Obesity_Risk_Category']]

# One-hot encode categorical features (Diet_Type and genetic variants)
genetic_df = pd.get_dummies(genetic_df, columns=['Diet_Type', 'MC4R_Variant', 'PPARG_Variant', 'FTO_Variant', 'LEPR_Variant'], drop_first=True)

# Prepare features and target
X = genetic_df.drop(['Obesity_Risk_Category'], axis=1)
y = genetic_df['Obesity_Risk_Category']

# Convert Obesity_Risk_Category to numeric labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Ensure all columns in X are numeric
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = pd.to_numeric(X[col], errors='coerce')  # Convert to numeric, coercing errors to NaN
        X[col] = X[col].fillna(0)  # Fill NaN values with 0

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y_encoded)

# Split resampled data into training (60%), validation (20%), and test (20%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X_resampled, y_resampled, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Hyperparameter tuning with GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],  # Limit tree depth
    'min_samples_split': [10, 20],  # Increase minimum samples to split
    'min_samples_leaf': [2, 4],  # Increase minimum samples per leaf
    'max_features': ['sqrt', 'log2']
}

# Use RandomForestClassifier for classification
rf = RandomForestClassifier(random_state=42, class_weight='balanced')
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(f'Best Parameters: {best_params}')

# Train the model with the best parameters
best_rf = grid_search.best_estimator_
best_rf.fit(X_train, y_train)

# Evaluate on the validation set
y_val_pred = best_rf.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred) * 100
print(f'Validation Accuracy: {val_accuracy:.2f}%')

# Evaluate on the test set
y_test_pred = best_rf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred) * 100
print(f'Test Accuracy: {test_accuracy:.2f}%')

# Evaluate on noisy data
X_noisy = X_resampled + np.random.normal(0, 0.1, X_resampled.shape)  # Add Gaussian noise
cv_scores_noisy = cross_val_score(best_rf, X_noisy, y_resampled, cv=5, scoring='accuracy')
print(f'Cross-Validation Accuracy (Noisy Data): {cv_scores_noisy.mean() * 100:.2f}%')

# Print classification report and confusion matrix
print("\nClassification Report (Test Set):")
print(classification_report(y_test, y_test_pred, target_names=label_encoder.classes_))

print("\nConfusion Matrix (Test Set):")
print(confusion_matrix(y_test, y_test_pred))

# Preprocess nutritional data for meal recommendations
nutritional_features = meal_df[['Energy_kcal', 'Protein_g', 'Fat_g', 'Carb_g']]
scaler = StandardScaler()
nutritional_features_scaled = scaler.fit_transform(nutritional_features)

# Increase the number of meal clusters
num_clusters = 10  # Use 10 clusters for more diversity
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
meal_df['Meal_Cluster'] = kmeans.fit_predict(nutritional_features_scaled)

# Function to display meal clusters in a user-friendly format
def display_meal_cluster(meals, cluster_name, num_meals=5):
    print(f"\n=== Recommended Meals for {cluster_name} ===")
    print(f"Total Meals in Cluster: {len(meals)}")
    
    # Display a subset of meals
    print(f"\nTop {num_meals} Meals:")
    print(f"{'Meal Description':<50} {'Energy (kcal)':<15} {'Protein (g)':<15} {'Fat (g)':<15} {'Carbs (g)':<15}")
    print("-" * 110)
    for _, meal in meals.head(num_meals).iterrows():
        print(f"{meal['Descrip']:<50} {meal['Energy_kcal']:<15} {meal['Protein_g']:<15} {meal['Fat_g']:<15} {meal['Carb_g']:<15}")
    
    # Display summary of nutritional values
    avg_energy = meals['Energy_kcal'].mean()
    avg_protein = meals['Protein_g'].mean()
    avg_fat = meals['Fat_g'].mean()
    avg_carbs = meals['Carb_g'].mean()
    
    print("\n=== Nutritional Summary ===")
    print(f"Average Energy (kcal): {avg_energy:.2f}")
    print(f"Average Protein (g): {avg_protein:.2f}")
    print(f"Average Fat (g): {avg_fat:.2f}")
    print(f"Average Carbs (g): {avg_carbs:.2f}")

# Function to recommend meals dynamically based on user profile
def recommend_meals(user_profile, meal_df, model, label_encoder, num_meals=5):
    # Convert the user profile to a DataFrame
    user_profile_df = pd.DataFrame([user_profile])
    
    # One-hot encode categorical features (same as during training)
    user_profile_df = pd.get_dummies(user_profile_df, columns=['Diet_Type', 'MC4R_Variant', 'PPARG_Variant', 'FTO_Variant', 'LEPR_Variant'], drop_first=True)
    
    # Ensure the user profile has the same columns as the training data
    missing_cols = set(X_train.columns) - set(user_profile_df.columns)
    for col in missing_cols:
        user_profile_df[col] = 0  # Add missing columns with default value 0
    
    # Reorder columns to match the training data
    user_profile_df = user_profile_df[X_train.columns]
    
    # Predict the obesity risk category for the user profile
    predicted_risk_category = model.predict(user_profile_df)
    predicted_risk_category_label = label_encoder.inverse_transform(predicted_risk_category)[0]
    print(f'Predicted Obesity Risk Category: {predicted_risk_category_label}')
    
    # Define cluster preferences based on risk category
    if predicted_risk_category_label == 'Low':
        # Low-risk users: Focus on high-protein, balanced meals
        preferred_clusters = [0, 1, 2]  # Example clusters
        sort_by = 'Protein_g'  # Sort by highest protein
    elif predicted_risk_category_label == 'Medium':
        # Medium-risk users: Focus on moderate-calorie, balanced meals
        preferred_clusters = [3, 4, 5]  # Example clusters
        sort_by = 'Energy_kcal'  # Sort by moderate calories
    else:
        # High-risk users: Focus on low-calorie, nutrient-dense meals
        preferred_clusters = [6, 7, 8, 9]  # Example clusters
        sort_by = 'Energy_kcal'  # Sort by lowest calories
    
    # Recommend meals from the preferred clusters
    recommended_meals = meal_df[meal_df['Meal_Cluster'].isin(preferred_clusters)]
    
    # Sort meals based on the user's risk category
    if predicted_risk_category_label == 'High':
        recommended_meals = recommended_meals.sort_values(by=sort_by, ascending=True)  # Low calories
    else:
        recommended_meals = recommended_meals.sort_values(by=sort_by, ascending=False)  # High protein or moderate calories
    
    # Display recommended meals in a user-friendly format
    display_meal_cluster(recommended_meals, f"{predicted_risk_category_label}-Risk", num_meals)

# Example new genetic profile
new_profile = {
    'BMI': 28.5,  # Example value
    'Physical_Activity': 2,  # Example value (1: Low, 2: Moderate, 3: High)
    'Diet_Type': 'High-Fat',  # Example value
    'MC4R_Variant': 'rs17782313_CT',  # Example value
    'PPARG_Variant': 'rs1801282_CG',  # Example value
    'FTO_Variant': 'rs9939609_AT',  # Example value
    'LEPR_Variant': 'rs1137101_AG',  # Example value
    'Obesity_Risk_Score': 0.45  # Example value
}

# Recommend meals for the new profile
recommend_meals(new_profile, meal_df, best_rf, label_encoder)

Best Parameters: {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 100}
Validation Accuracy: 100.00%
Test Accuracy: 100.00%
Cross-Validation Accuracy (Noisy Data): 82.86%

Classification Report (Test Set):
              precision    recall  f1-score   support

        High       1.00      1.00      1.00       182
         Low       1.00      1.00      1.00       154
      Medium       1.00      1.00      1.00       173

    accuracy                           1.00       509
   macro avg       1.00      1.00      1.00       509
weighted avg       1.00      1.00      1.00       509


Confusion Matrix (Test Set):
[[182   0   0]
 [  0 154   0]
 [  0   0 173]]
Predicted Obesity Risk Category: Medium

=== Recommended Meals for Medium-Risk ===
Total Meals in Cluster: 3637

Top 5 Meals:
Meal Description                                   Energy (kcal)   Protein (g)     Fat (g)         Carbs (g)      
-----------------------------------------

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Load dataset
genetic_df = pd.read_csv('C:\\Users\\trejan\\Desktop\\Sem 2\\Machine Learning\\model\\genetic_profiles.csv')

# One-Hot Encoding for categorical variables
categorical_cols = ['MC4R_Variant', 'PPARG_Variant', 'FTO_Variant', 'LEPR_Variant', 'Physical_Activity']
genetic_df = pd.get_dummies(genetic_df, columns=categorical_cols)

# Separate features and target
X = genetic_df.drop(columns=['Profile_ID', 'Diet_Type'])  # Removing Profile_ID & target

y = genetic_df['Diet_Type'].astype('category').cat.codes  # Convert target to numerical labels

# Feature Scaling
scaler = StandardScaler()
X[['Age', 'BMI']] = scaler.fit_transform(X[['Age', 'BMI']])

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define model with class weights
model = RandomForestClassifier(class_weight='balanced', random_state=42)

# Hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = RandomizedSearchCV(model, param_grid, n_iter=10, cv=5, scoring='accuracy', random_state=42)
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Predict
y_pred = best_model.predict(X_test)

# Evaluation Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Model Performance:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}\n")

# Print Classification Report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Display some sample predictions
sample_preds = pd.DataFrame({'Actual': y_test[:10].values, 'Predicted': y_pred[:10]})
print("\nSample Predictions:")
print(sample_preds)


Model Performance:
Accuracy: 0.3550
Precision: 0.3552
Recall: 0.3550
F1 Score: 0.3547

Classification Report:
              precision    recall  f1-score   support

           0       0.37      0.36      0.36       144
           1       0.35      0.38      0.36       130
           2       0.35      0.33      0.34       126

    accuracy                           0.35       400
   macro avg       0.35      0.35      0.35       400
weighted avg       0.36      0.35      0.35       400


Sample Predictions:
   Actual  Predicted
0       0          2
1       0          2
2       0          1
3       1          2
4       0          1
5       2          1
6       0          1
7       0          1
8       2          0
9       1          1


In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.cluster import KMeans
from imblearn.over_sampling import SMOTE

# Load datasets
genetic_df = pd.read_csv("C:\\Users\\trejan\\Desktop\\Sem 2\\Machine Learning\\model\\new_genetic_profiles.csv")
meal_df = pd.read_csv("C:\\Users\\trejan\\Desktop\\Sem 2\\Machine Learning\\model\\train.csv")

# Handle missing values for numeric columns in genetic_df
numeric_cols = genetic_df.select_dtypes(include=['float64', 'int64']).columns
genetic_df[numeric_cols] = genetic_df[numeric_cols].fillna(genetic_df[numeric_cols].mean())

# Handle missing values for categorical columns in genetic_df
categorical_cols = genetic_df.select_dtypes(include=['object']).columns
genetic_df[categorical_cols] = genetic_df[categorical_cols].fillna('Unknown')

# Convert Obesity_Risk_Score into categories (low, medium, high)
genetic_df['Obesity_Risk_Category'] = pd.cut(
    genetic_df['Obesity_Risk_Score'],
    bins=[0, 0.3, 0.6, 1],
    labels=['Low', 'Medium', 'High']
)

# Map gene variants to numerical values based on their functional impact
variant_mapping = {
    'MC4R_Variant': {'rs17782313_CT': 0.5, 'rs17782313_TT': 1.0, 'rs17782313_CC': 0.0, 'None': 0},
    'PPARG_Variant': {'rs1801282_CG': 0.3, 'rs1801282_GG': 0.7, 'rs1801282_CC': 0.0, 'None': 0},
    'FTO_Variant': {'rs9939609_AT': 0.4, 'rs9939609_TT': 0.8, 'rs9939609_AA': 0.0, 'None': 0},
    'LEPR_Variant': {'rs1137101_AG': 0.6, 'rs1137101_GG': 0.9, 'rs1137101_AA': 0.0, 'None': 0}
}

for variant, mapping in variant_mapping.items():
    genetic_df[variant] = genetic_df[variant].map(mapping)

# Create a genetic risk score
genetic_df['Genetic_Risk_Score'] = (
    genetic_df['MC4R_Present'] * genetic_df['MC4R_Variant'] +
    genetic_df['PPARG_Present'] * genetic_df['PPARG_Variant'] +
    genetic_df['FTO_Present'] * genetic_df['FTO_Variant'] +
    genetic_df['LEPR_Present'] * genetic_df['LEPR_Variant']
)

# Create interaction features
genetic_df['BMI_MC4R_Interaction'] = genetic_df['BMI'] * genetic_df['MC4R_Present']
genetic_df['Physical_Activity_FTO_Interaction'] = genetic_df['Physical_Activity'] * genetic_df['FTO_Present']

# Select relevant features
features = [
    'BMI', 'Physical_Activity', 'Diet_Type', 'Obesity_Risk_Score',
    'MC4R_Present', 'PPARG_Present', 'FTO_Present', 'LEPR_Present',
    'Genetic_Risk_Score', 'BMI_MC4R_Interaction', 'Physical_Activity_FTO_Interaction'
]
genetic_df = genetic_df[features + ['Obesity_Risk_Category']]

# One-hot encode only Diet_Type (not genetic variants)
genetic_df = pd.get_dummies(genetic_df, columns=['Diet_Type'], drop_first=True)

# Prepare features and target
X = genetic_df.drop(['Obesity_Risk_Category'], axis=1)
y = genetic_df['Obesity_Risk_Category']

# Convert Obesity_Risk_Category to numeric labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Ensure all columns in X are numeric
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = pd.to_numeric(X[col], errors='coerce')  # Convert to numeric, coercing errors to NaN
        X[col] = X[col].fillna(0)  # Fill NaN values with 0

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y_encoded)

# Split resampled data into training (60%), validation (20%), and test (20%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X_resampled, y_resampled, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Hyperparameter tuning with GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],  # Limit tree depth
    'min_samples_split': [10, 20],  # Increase minimum samples to split
    'min_samples_leaf': [2, 4],  # Increase minimum samples per leaf
    'max_features': ['sqrt', 'log2']
}

# Use RandomForestClassifier for classification
rf = RandomForestClassifier(random_state=42, class_weight='balanced')
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(f'Best Parameters: {best_params}')

# Train the model with the best parameters
best_rf = grid_search.best_estimator_
best_rf.fit(X_train, y_train)

# Evaluate on the validation set
y_val_pred = best_rf.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred) * 100
print(f'Validation Accuracy: {val_accuracy:.2f}%')

# Evaluate on the test set
y_test_pred = best_rf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred) * 100
print(f'Test Accuracy: {test_accuracy:.2f}%')

# Evaluate on noisy data
X_noisy = X_resampled + np.random.normal(0, 0.1, X_resampled.shape)  # Add Gaussian noise
cv_scores_noisy = cross_val_score(best_rf, X_noisy, y_resampled, cv=5, scoring='accuracy')
print(f'Cross-Validation Accuracy (Noisy Data): {cv_scores_noisy.mean() * 100:.2f}%')

# Print classification report and confusion matrix
print("\nClassification Report (Test Set):")
print(classification_report(y_test, y_test_pred, target_names=label_encoder.classes_))

print("\nConfusion Matrix (Test Set):")
print(confusion_matrix(y_test, y_test_pred))

# Preprocess nutritional data for meal recommendations
nutritional_features = meal_df[['Energy_kcal', 'Protein_g', 'Fat_g', 'Carb_g']]
scaler = StandardScaler()
nutritional_features_scaled = scaler.fit_transform(nutritional_features)

# Increase the number of meal clusters
num_clusters = 10  # Use 10 clusters for more diversity
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
meal_df['Meal_Cluster'] = kmeans.fit_predict(nutritional_features_scaled)

# Function to display meal clusters in a user-friendly format
def display_meal_cluster(meals, cluster_name, num_meals=5):
    print(f"\n=== Recommended Meals for {cluster_name} ===")
    print(f"Total Meals in Cluster: {len(meals)}")
    
    # Display a subset of meals
    print(f"\nTop {num_meals} Meals:")
    print(f"{'Meal Description':<50} {'Energy (kcal)':<15} {'Protein (g)':<15} {'Fat (g)':<15} {'Carbs (g)':<15}")
    print("-" * 110)
    for _, meal in meals.head(num_meals).iterrows():
        print(f"{meal['Descrip']:<50} {meal['Energy_kcal']:<15} {meal['Protein_g']:<15} {meal['Fat_g']:<15} {meal['Carb_g']:<15}")
    
    # Display summary of nutritional values
    avg_energy = meals['Energy_kcal'].mean()
    avg_protein = meals['Protein_g'].mean()
    avg_fat = meals['Fat_g'].mean()
    avg_carbs = meals['Carb_g'].mean()
    
    print("\n=== Nutritional Summary ===")
    print(f"Average Energy (kcal): {avg_energy:.2f}")
    print(f"Average Protein (g): {avg_protein:.2f}")
    print(f"Average Fat (g): {avg_fat:.2f}")
    print(f"Average Carbs (g): {avg_carbs:.2f}")

# Function to recommend meals dynamically based on user profile
def recommend_meals(user_profile, meal_df, model, label_encoder, num_meals=5):
    # Convert the user profile to a DataFrame
    user_profile_df = pd.DataFrame([user_profile])
    
    # One-hot encode categorical features (same as during training)
    user_profile_df = pd.get_dummies(user_profile_df, columns=['Diet_Type'], drop_first=True)
    
    # Ensure the user profile has the same columns as the training data
    missing_cols = set(X_train.columns) - set(user_profile_df.columns)
    for col in missing_cols:
        user_profile_df[col] = 0  # Add missing columns with default value 0
    
    # Reorder columns to match the training data
    user_profile_df = user_profile_df[X_train.columns]
    
    # Predict the obesity risk category for the user profile
    predicted_risk_category = model.predict(user_profile_df)
    predicted_risk_category_label = label_encoder.inverse_transform(predicted_risk_category)[0]
    print(f'Predicted Obesity Risk Category: {predicted_risk_category_label}')
    
    # Define cluster preferences based on risk category
    if predicted_risk_category_label == 'Low':
        # Low-risk users: Focus on high-protein, balanced meals
        preferred_clusters = [0, 1, 2]  # Example clusters
        sort_by = 'Protein_g'  # Sort by highest protein
    elif predicted_risk_category_label == 'Medium':
        # Medium-risk users: Focus on moderate-calorie, balanced meals
        preferred_clusters = [3, 4, 5]  # Example clusters
        sort_by = 'Energy_kcal'  # Sort by moderate calories
    else:
        # High-risk users: Focus on low-calorie, nutrient-dense meals
        preferred_clusters = [6, 7, 8, 9]  # Example clusters
        sort_by = 'Energy_kcal'  # Sort by lowest calories
    
    # Recommend meals from the preferred clusters
    recommended_meals = meal_df[meal_df['Meal_Cluster'].isin(preferred_clusters)]
    
    # Sort meals based on the user's risk category
    if predicted_risk_category_label == 'High':
        recommended_meals = recommended_meals.sort_values(by=sort_by, ascending=True)  # Low calories
    else:
        recommended_meals = recommended_meals.sort_values(by=sort_by, ascending=False)  # High protein or moderate calories
    
    # Display recommended meals in a user-friendly format
    display_meal_cluster(recommended_meals, f"{predicted_risk_category_label}-Risk", num_meals)

# Example new genetic profile
new_profile = {
    'BMI': 28.5,  # Example value
    'Physical_Activity': 2,  # Example value (1: Low, 2: Moderate, 3: High)
    'Diet_Type': 'High-Fat',  # Example value
    'Obesity_Risk_Score': 0.45,  # Example value
    'MC4R_Present': 1,  # Example value
    'PPARG_Present': 0,  # Example value
    'FTO_Present': 1,  # Example value
    'LEPR_Present': 1  # Example value
}

# Recommend meals for the new profile
recommend_meals(new_profile, meal_df, best_rf, label_encoder)

ValueError: Input X contains NaN.
SMOTE does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [8]:
print(X.isnull().sum())

BMI                                     0
Physical_Activity                       0
Obesity_Risk_Score                      0
MC4R_Present                            0
PPARG_Present                           0
FTO_Present                             0
LEPR_Present                            0
Genetic_Risk_Score                   1469
BMI_MC4R_Interaction                    0
Physical_Activity_FTO_Interaction       0
Diet_Type_High-Carb                     0
Diet_Type_High-Fat                      0
dtype: int64


In [11]:
print(X.isnull().sum())


BMI                                     0
Physical_Activity                       0
Obesity_Risk_Score                      0
MC4R_Present                            0
PPARG_Present                           0
FTO_Present                             0
LEPR_Present                            0
Genetic_Risk_Score                   1469
BMI_MC4R_Interaction                    0
Physical_Activity_FTO_Interaction       0
Diet_Type_High-Carb                     0
Diet_Type_High-Fat                      0
dtype: int64


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.cluster import KMeans
from imblearn.over_sampling import SMOTE

# Load datasets
genetic_df = pd.read_csv("C:\\Users\\trejan\\Desktop\\Sem 2\\Machine Learning\\model\\new_genetic_profiles.csv")
meal_df = pd.read_csv("C:\\Users\\trejan\\Desktop\\Sem 2\\Machine Learning\\model\\train.csv")

# Handle missing values for numeric columns in genetic_df
numeric_cols = genetic_df.select_dtypes(include=['float64', 'int64']).columns
genetic_df[numeric_cols] = genetic_df[numeric_cols].fillna(genetic_df[numeric_cols].mean())

# Handle missing values for categorical columns in genetic_df
categorical_cols = genetic_df.select_dtypes(include=['object']).columns
genetic_df[categorical_cols] = genetic_df[categorical_cols].fillna('Unknown')

# Convert Obesity_Risk_Score into categories (low, medium, high)
genetic_df['Obesity_Risk_Category'] = pd.cut(
    genetic_df['Obesity_Risk_Score'],
    bins=[0, 0.3, 0.6, 1],
    labels=['Low', 'Medium', 'High']
)

# Calculate Genetic_Risk_Score using only presence/absence columns
genetic_df['Genetic_Risk_Score'] = (
    genetic_df['MC4R_Present'] +
    genetic_df['PPARG_Present'] +
    genetic_df['FTO_Present'] +
    genetic_df['LEPR_Present']
)

# Create interaction features
genetic_df['BMI_MC4R_Interaction'] = genetic_df['BMI'] * genetic_df['MC4R_Present']
genetic_df['Physical_Activity_FTO_Interaction'] = genetic_df['Physical_Activity'] * genetic_df['FTO_Present']

# Select relevant features
features = [
    'BMI', 'Physical_Activity', 'Diet_Type', 'Obesity_Risk_Score',
    'MC4R_Present', 'PPARG_Present', 'FTO_Present', 'LEPR_Present',
    'Genetic_Risk_Score', 'BMI_MC4R_Interaction', 'Physical_Activity_FTO_Interaction'
]
genetic_df = genetic_df[features + ['Obesity_Risk_Category']]

# One-hot encode only Diet_Type (not genetic variants)
genetic_df = pd.get_dummies(genetic_df, columns=['Diet_Type'], drop_first=True)

# Prepare features and target
X = genetic_df.drop(['Obesity_Risk_Category'], axis=1)
y = genetic_df['Obesity_Risk_Category']

# Convert Obesity_Risk_Category to numeric labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Ensure all columns in X are numeric
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = pd.to_numeric(X[col], errors='coerce')  # Convert to numeric, coercing errors to NaN
        X[col] = X[col].fillna(0)  # Fill NaN values with 0

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y_encoded)

# Split resampled data into training (60%), validation (20%), and test (20%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X_resampled, y_resampled, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Hyperparameter tuning with GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],  # Limit tree depth
    'min_samples_split': [10, 20],  # Increase minimum samples to split
    'min_samples_leaf': [2, 4],  # Increase minimum samples per leaf
    'max_features': ['sqrt', 'log2']
}

# Use RandomForestClassifier for classification
rf = RandomForestClassifier(random_state=42, class_weight='balanced')
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(f'Best Parameters: {best_params}')

# Train the model with the best parameters
best_rf = grid_search.best_estimator_
best_rf.fit(X_train, y_train)

# Evaluate on the validation set
y_val_pred = best_rf.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred) * 100
print(f'Validation Accuracy: {val_accuracy:.2f}%')

# Evaluate on the test set
y_test_pred = best_rf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred) * 100
print(f'Test Accuracy: {test_accuracy:.2f}%')

# Evaluate on noisy data
X_noisy = X_resampled + np.random.normal(0, 0.1, X_resampled.shape)  # Add Gaussian noise
cv_scores_noisy = cross_val_score(best_rf, X_noisy, y_resampled, cv=5, scoring='accuracy')
print(f'Cross-Validation Accuracy (Noisy Data): {cv_scores_noisy.mean() * 100:.2f}%')

# Print classification report and confusion matrix
print("\nClassification Report (Test Set):")
print(classification_report(y_test, y_test_pred, target_names=label_encoder.classes_))

print("\nConfusion Matrix (Test Set):")
print(confusion_matrix(y_test, y_test_pred))

# Preprocess nutritional data for meal recommendations
nutritional_features = meal_df[['Energy_kcal', 'Protein_g', 'Fat_g', 'Carb_g']]
scaler = StandardScaler()
nutritional_features_scaled = scaler.fit_transform(nutritional_features)

# Increase the number of meal clusters
num_clusters = 10  # Use 10 clusters for more diversity
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
meal_df['Meal_Cluster'] = kmeans.fit_predict(nutritional_features_scaled)

# Function to display meal clusters in a user-friendly format
def display_meal_cluster(meals, cluster_name, num_meals=5):
    print(f"\n=== Recommended Meals for {cluster_name} ===")
    print(f"Total Meals in Cluster: {len(meals)}")
    
    # Display a subset of meals
    print(f"\nTop {num_meals} Meals:")
    print(f"{'Meal Description':<50} {'Energy (kcal)':<15} {'Protein (g)':<15} {'Fat (g)':<15} {'Carbs (g)':<15}")
    print("-" * 110)
    for _, meal in meals.head(num_meals).iterrows():
        print(f"{meal['Descrip']:<50} {meal['Energy_kcal']:<15} {meal['Protein_g']:<15} {meal['Fat_g']:<15} {meal['Carb_g']:<15}")
    
    # Display summary of nutritional values
    avg_energy = meals['Energy_kcal'].mean()
    avg_protein = meals['Protein_g'].mean()
    avg_fat = meals['Fat_g'].mean()
    avg_carbs = meals['Carb_g'].mean()
    
    print("\n=== Nutritional Summary ===")
    print(f"Average Energy (kcal): {avg_energy:.2f}")
    print(f"Average Protein (g): {avg_protein:.2f}")
    print(f"Average Fat (g): {avg_fat:.2f}")
    print(f"Average Carbs (g): {avg_carbs:.2f}")

# Function to recommend meals dynamically based on user profile
def recommend_meals(user_profile, meal_df, model, label_encoder, num_meals=5):
    # Convert the user profile to a DataFrame
    user_profile_df = pd.DataFrame([user_profile])
    
    # One-hot encode categorical features (same as during training)
    user_profile_df = pd.get_dummies(user_profile_df, columns=['Diet_Type'], drop_first=True)
    
    # Ensure the user profile has the same columns as the training data
    missing_cols = set(X_train.columns) - set(user_profile_df.columns)
    for col in missing_cols:
        user_profile_df[col] = 0  # Add missing columns with default value 0
    
    # Reorder columns to match the training data
    user_profile_df = user_profile_df[X_train.columns]
    
    # Predict the obesity risk category for the user profile
    predicted_risk_category = model.predict(user_profile_df)
    predicted_risk_category_label = label_encoder.inverse_transform(predicted_risk_category)[0]
    print(f'Predicted Obesity Risk Category: {predicted_risk_category_label}')
    
    # Define cluster preferences based on risk category
    if predicted_risk_category_label == 'Low':
        # Low-risk users: Focus on high-protein, balanced meals
        preferred_clusters = [0, 1, 2]  # Example clusters
        sort_by = 'Protein_g'  # Sort by highest protein
    elif predicted_risk_category_label == 'Medium':
        # Medium-risk users: Focus on moderate-calorie, balanced meals
        preferred_clusters = [3, 4, 5]  # Example clusters
        sort_by = 'Energy_kcal'  # Sort by moderate calories
    else:
        # High-risk users: Focus on low-calorie, nutrient-dense meals
        preferred_clusters = [6, 7, 8, 9]  # Example clusters
        sort_by = 'Energy_kcal'  # Sort by lowest calories
    
    # Recommend meals from the preferred clusters
    recommended_meals = meal_df[meal_df['Meal_Cluster'].isin(preferred_clusters)]
    
    # Sort meals based on the user's risk category
    if predicted_risk_category_label == 'High':
        recommended_meals = recommended_meals.sort_values(by=sort_by, ascending=True)  # Low calories
    else:
        recommended_meals = recommended_meals.sort_values(by=sort_by, ascending=False)  # High protein or moderate calories
    
    # Display recommended meals in a user-friendly format
    display_meal_cluster(recommended_meals, f"{predicted_risk_category_label}-Risk", num_meals)

# Example new genetic profile
new_profile = {
    'BMI': 28.5,  # Example value
    'Physical_Activity': 3,  # Example value (1: Low, 2: Moderate, 3: High)
    'Diet_Type': 'High-Fat',  # Example value
    'Obesity_Risk_Score': 0.8,  # Example value
    'MC4R_Present': 1,  # Example value
    'PPARG_Present': 0,  # Example value
    'FTO_Present': 1,  # Example value
    'LEPR_Present': 0  # Example value
}

# Recommend meals for the new profile
recommend_meals(new_profile, meal_df, best_rf, label_encoder)

Best Parameters: {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 100}
Validation Accuracy: 100.00%
Test Accuracy: 100.00%
Cross-Validation Accuracy (Noisy Data): 87.03%

Classification Report (Test Set):
              precision    recall  f1-score   support

        High       1.00      1.00      1.00       194
         Low       1.00      1.00      1.00       202
      Medium       1.00      1.00      1.00       167

    accuracy                           1.00       563
   macro avg       1.00      1.00      1.00       563
weighted avg       1.00      1.00      1.00       563


Confusion Matrix (Test Set):
[[194   0   0]
 [  0 202   0]
 [  0   0 167]]


C:\Users\trejan\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\trejan\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "C:\Users\trejan\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 554, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\trejan\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 

Predicted Obesity Risk Category: High

=== Recommended Meals for High-Risk ===
Total Meals in Cluster: 1941

Top 5 Meals:
Meal Description                                   Energy (kcal)   Protein (g)     Fat (g)         Carbs (g)      
--------------------------------------------------------------------------------------------------------------
Sweetener, herbal extract powder from Stevia leaf  0.0             0.0             0.0             100.0          
Turkey, breast, from whole bird, enhanced, meat only, roasted 127.0           26.97           2.08            0.0            
Turkey from whole, enhanced, light meat, meat only, cooked, roasted 127.0           26.97           2.08            0.0            
Turkey, wing, from whole bird, enhanced, meat only, roasted 127.0           26.97           2.08            0.0            
Game meat, buffalo, water, cooked, roasted         131.0           26.83           1.8             0.0            

=== Nutritional Summary ===
Average Ene